In [18]:
system_prompt ='''
请根据用户提供的自然语言描述生成一个MongoDB的查询对象，除非用户明确指出待查询的关键字为精确匹配，否则查询对象应支持模糊匹配。
用户输入的内容放置在<input></input>标签中，根据以下步骤生成查询对象：
1. 首先识别用户是否是询问了关于某个问题，如“查找项目名称包含‘***’的记录”、“查找可能设备名称‘****’的信息”等，并提取关键词。
2. 如果用户提出的不是相关查询信息，则询问用户是否需要进一步说明，如“请问您要查找哪个项目的设备信息？”、“请问您要查找哪个设备的采购信息？”等。
3. 如果识别到关键字，将关键字进行提取并按照<requirements></requirements>中的要求输出符合mongodb查询语法的查询对象，最后将完整的查询对象输出在<query></query>标签中。
<requirements>
目标集合的名称为`equipment_collection`包含以下字段：`project_name`（字符串，项目名称）、`contract_number`（字符串，合同编号）、`contract_type`（字符串，合同类型）、\
`subitem_name`（字符串，子项名称）、`device_name`（字符串，设备名称）、`specification_material`（字符串，规格材质）、\
`manufacturer`（字符串，制造商）。
用户的需求是：查找与输入关键词相关的所有记录，关键词可能出现在上述任意字段中。
如果是模糊匹配要求（大小写不敏感），覆盖所有可能的数据，查询结果包含所有字段。
注意不要虚构任何数据并按照用户的需求进行查询。
构建的查询语句应该以`db.equipment_collection.find(${query_json})`为模板。
只需要提供完整的MongoDB查询对象`query_json`，输出在<query></query>标签中，并确保语法正确。
</requirements>
'''

In [19]:
from ollama import Client
client = Client(
  host='http://192.168.43.41:11434',
  headers={'x-some-header': 'some-value'}
)
response = client.chat(model='qwq:latest', 
                       options={
                           'temperature':0,
                           "num_ctx": 4096,
                       },
                       messages=[
  {'role':'system', 'content': system_prompt},
  {
    'role': 'user',
    'content': f"<input>BGYRHS-2023-14有哪些哪些设备？</input>"+"<think>\n",
  },
])
print(response.message.content)

input_text = response.message.content

<think>
好的，我需要处理用户的查询：“BGYRHS-2023-14有哪些哪些设备？”。首先，用户的问题是关于查找特定合同编号下的设备信息。根据要求，我要生成MongoDB的查询对象。

首先，确定用户是否在询问查询相关信息。这里明显是在问设备，所以需要提取关键词。合同编号是“BGYRHS-2023-14”，用户想知道这个合同下的设备。根据需求，除非明确要精确匹配，否则支持模糊匹配。但合同编号通常是精确的，用户可能希望准确匹配。

接下来，检查目标集合和字段。集合是equipment_collection，相关字段包括contract_number。用户需要查找该合同号的所有记录，并获取所有字段。因此查询条件应为contract_number等于“BGYRHS-2023-14”。

由于用户没有提到模糊匹配的需求，这里应该用精确匹配。所以查询对象应该是{ "contract_number": "BGYRHS-2023-14" }。

确认是否覆盖所有可能字段：是的，因为只需要合同编号匹配，其他字段会自然包含在结果中。不需要虚构数据，符合用户需求。
</think>

<query>{"contract_number": "BGYRHS-2023-14"}</query>


In [41]:
import re
import json

def extract_last_query_content(text):
    pattern = r'<query>(.*?)</query>'
    matches = list(re.finditer(pattern, text, re.DOTALL))
    if matches:
        return matches[-1].group(1)  # 返回最后一个匹配的捕获组
    return None

In [42]:
query_text = extract_last_query_content(input_text)

In [43]:
query_text

'{"contract_number": "BGYRHS-2023-14"}'

In [ ]:
from pymongo import MongoClient


# 连接到 MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['equipment_db']  # 数据库名

# 将字符串转换为字典
query_dict = json.loads(query_text)

# 执行查询
results = db.equipment_collection.find(query_dict)

# 打印结果
print("找到的文档数量:", len(list(results)))


找到的文档数量: 75
